In [27]:
import numpy as np
import scipy 
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.optimize import minimize
import pandas as pd


In [28]:
# Load the dataset
df = pd.read_csv("/Users/edoardodicosimo/Documents/Magistrale/ctfme/current.csv")

# Clean the DataFrame by removing the row with transformation codes
df_cleaned = df.drop(index=0)
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned['sasdate'] = pd.to_datetime(df_cleaned['sasdate'], format='%m/%d/%Y')

# Extract transformation codes
transformation_codes = df.iloc[0, 1:].to_frame().reset_index()
transformation_codes.columns = ['Series', 'Transformation_Code']

# Function to apply transformations based on the transformation code
def apply_transformation(series, code):
    if code == 1:
        # No transformation
        return series
    elif code == 2:
        # First difference
        return series.diff()
    elif code == 3:
        # Second difference
        return series.diff().diff()
    elif code == 4:
        # Log
        return np.log(series)
    elif code == 5:
        # First difference of log
        return np.log(series).diff()
    elif code == 6:
        # Second difference of log
        return np.log(series).diff().diff()
    elif code == 7:
        # Delta (x_t/x_{t-1} - 1)
        return series.pct_change()
    else:
        raise ValueError("Invalid transformation code")

# Applying the transformations to each column in df_cleaned based on transformation_codes
for series_name, code in transformation_codes.values:
    df_cleaned[series_name] = apply_transformation(df_cleaned[series_name].astype(float), float(code))


df_cleaned = df_cleaned[2:]
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.head(500)
# Extract transformation codes

,sasdate,RPI,W875RX1,DPCERA3M086SBEA,CMRMTSPLx,RETAILx,INDPRO,IPFPNSS,IPFINAL,IPCONGD,...,DNDGRG3M086SBEA,DSERRG3M086SBEA,CES0600000008,CES2000000008,CES3000000008,UMCSENTx,DTCOLNVHFNM,DTCTHFNM,INVEST,VIXCLSx
0,1959-03-01,0.006457,0.007325,0.009404,-0.003374,0.008321,0.014306,0.006035,0.004894,0.000000,...,-0.001148,0.000292,-0.000022,-0.008147,0.004819,NaN,0.004929,0.004138,-0.014792,NaN
1,1959-04-01,0.006510,0.007029,-0.003622,0.019915,0.000616,0.021075,0.014338,0.014545,0.015650,...,0.001312,0.001760,-0.000022,0.012203,-0.004890,NaN,0.012134,0.006734,0.024929,NaN
2,1959-05-01,0.005796,0.006618,0.012043,0.006839,0.007803,0.014955,0.008270,0.009582,0.004770,...,-0.001695,-0.001867,-0.000021,-0.004090,-0.004819,NaN,0.002828,0.002020,-0.015342,NaN
3,1959-06-01,0.003068,0.003012,0.003642,-0.000097,0.009064,0.001141,0.007034,0.007128,-0.004767,...,0.003334,0.001946,-0.004619,0.003992,0.004796,NaN,0.009726,0.009007,-0.012252,NaN
4,1959-07-01,-0.000580,-0.000762,-0.003386,0.012155,-0.000330,-0.024240,0.001168,0.008249,0.013054,...,-0.001204,-0.000013,0.000000,-0.004040,-0.004796,NaN,-0.004631,-0.001000,0.029341,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2000-06-01,0.002845,0.004783,0.002123,0.006589,0.007445,0.000753,0.000291,0.001276,0.000805,...,0.010881,0.000796,0.003291,0.001150,0.006313,-4.3,-0.006503,0.008705,0.000859,24.3177
496,2000-07-01,0.005314,0.005874,0.000697,-0.006322,-0.002901,-0.001741,0.000318,-0.000065,-0.004127,...,-0.006584,0.000683,-0.000011,0.004587,-0.003514,1.9,0.014152,-0.001007,0.002365,22.6600
497,2000-08-01,0.005457,0.005257,0.005126,0.000347,0.001056,-0.002681,-0.005068,-0.005411,-0.004769,...,-0.007677,-0.000775,-0.000011,-0.001165,-0.000004,-1.0,-0.016017,-0.002497,-0.009325,20.4843
498,2000-09-01,0.000920,0.000980,0.008228,0.010731,0.016823,0.003967,0.005414,0.007476,0.009292,...,0.014826,0.000344,-0.000011,-0.000016,0.000690,-0.5,-0.007393,0.006676,0.006768,22.3365


In [29]:
def unconditional_ar_mean_variance(c, phis, sigma2):
    ## The length of phis is p
    p = len(phis)
    A = np.zeros((p, p))
    A[0, :] = phis
    A[1:, 0:(p-1)] = np.eye(p-1)
    ## Check for stationarity
    eigA = np.linalg.eig(A)
    if all(np.abs(eigA.eigenvalues)<1):
        stationary = True
    else:
        stationary = False
    # Create the vector b
    b = np.zeros((p, 1))
    b[0, 0] = c
    
    # Compute the mean using matrix algebra
    I = np.eye(p)
    mu = np.linalg.inv(I - A) @ b
    
    # Solve the discrete Lyapunov equation
    Q = np.zeros((p, p))
    Q[0, 0] = sigma2
    #Sigma = np.linalg.solve(I - np.kron(A, A), Q.flatten()).reshape(7, 7)
    Sigma = scipy.linalg.solve_discrete_lyapunov(A, Q)
    
    return mu.ravel(), Sigma, stationary


In [30]:
def lagged_matrix(Y, max_lag=7):
    n = len(Y)
    lagged_matrix = np.full((n, max_lag), np.nan)    
    # Fill each column with the appropriately lagged data
    for lag in range(1, max_lag + 1):
        lagged_matrix[lag:, lag - 1] = Y[:-lag]
    return lagged_matrix

In [31]:
def cond_loglikelihood_ar7(params, y):
    
    c = params[0] 
    phi = params[1:8]
    sigma2 = params[8]
    
    mu, Sigma, stationary = unconditional_ar_mean_variance(c, phi, sigma2) 
    ## We could check that at phis the process is stationary and return -Inf if it is not
    if not(stationary):
        return -np.inf
    ## The distribution of 
    # y_t|y_{t-1}, ..., y_{t-7} ~ N(c+\phi_{1}*y_{t-1}+...+\phi_{7}y_{t-7}, sigma2)
    ## Create lagged matrix
    X = lagged_matrix(y, 7)
    yf = y[7:]
    Xf = X[7:,:]
    loglik = np.sum(norm.logpdf(yf, loc=(c + Xf@phi), scale=np.sqrt(sigma2)))
    return loglik

In [32]:
def uncond_loglikelihood_ar7(params, y):
    ## The unconditional loglikelihood
    ## is the unconditional "plus" the density of the
    ## first p (7 in our case) observations
    cloglik = cond_loglikelihood_ar7(params, y)

    ## Calculate initial
    # y_1, ..., y_7 ~ N(mu, sigma_y)
    c = params[0] 
    phi = params[1:8]
    sigma2 = params[8]
    mu, Sigma, stationary = unconditional_ar_mean_variance(c, phi, sigma2)
    if not(stationary):
        return -np.inf
    mvn = multivariate_normal(mean=mu, cov=Sigma, allow_singular=True)
    uloglik = cloglik + mvn.logpdf(y[0:7])
    return uloglik

In [33]:
y = df_cleaned['INDPRO']
X = lagged_matrix(y, 7)
yf = y[7:]
Xf = np.hstack((np.ones((len(y)-7,1)), X[7:,:]))
beta = np.linalg.solve(Xf.T@Xf, Xf.T@yf)
sigma2_hat = np.mean((yf - Xf@beta)**2)

params = np.hstack((beta, sigma2_hat))
print(params)

[ 1.25663978e-03  2.93198980e-01 -7.34894401e-02  4.67307848e-02
  4.70762950e-02 -2.59173496e-02  6.13114483e-02  1.91136675e-02
  8.88022999e-05]


In [34]:

def cobj(params, y): 
    return - cond_loglikelihood_ar7(params,y)

cond_estimate = scipy.optimize.minimize(cobj, params, args = y, method='Nelder-Mead')
print(cond_estimate)

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -2505.6112836659663
             x: [ 1.257e-03  2.932e-01 -7.349e-02  4.673e-02  4.708e-02
                 -2.592e-02  6.131e-02  1.911e-02  8.880e-05]
           nit: 91
          nfev: 149
 final_simplex: (array([[ 1.257e-03,  2.932e-01, ...,  1.911e-02,
                         8.880e-05],
                       [ 1.257e-03,  2.932e-01, ...,  1.912e-02,
                         8.880e-05],
                       ...,
                       [ 1.256e-03,  2.933e-01, ...,  1.907e-02,
                         8.881e-05],
                       [ 1.256e-03,  2.933e-01, ...,  1.906e-02,
                         8.879e-05]]), array([-2.506e+03, -2.506e+03, -2.506e+03, -2.506e+03,
                       -2.506e+03, -2.506e+03, -2.506e+03, -2.506e+03,
                       -2.506e+03, -2.506e+03]))


In [35]:
def uobj(params, y): 
    return - uncond_loglikelihood_ar7(params,y)

uncond_estimate = scipy.optimize.minimize(uobj, cond_estimate.x, args = y, method='L-BFGS-B')
print(uncond_estimate)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: -2520.0742854199407
        x: [ 1.236e-03  2.932e-01 -7.349e-02  4.673e-02  4.708e-02
            -2.592e-02  6.131e-02  1.911e-02  9.068e-05]
      nit: 12
      jac: [-2.308e+00 -1.018e+01  4.549e+00  1.075e+01  9.498e+00
             3.035e+00 -7.585e-01 -5.969e-01 -9.158e+00]
     nfev: 210
     njev: 21
 hess_inv: <9x9 LbfgsInvHessProduct with dtype=float64>


In [36]:
bounds_constant = tuple((-np.inf, np.inf) for _ in range(1))
bounds_phi = tuple((-1, 1) for _ in range(7))
bounds_sigma = tuple((0,np.inf) for _ in range(1))
bounds = bounds_constant + bounds_phi + bounds_sigma

In [37]:

def calculate_forecast(params,y,h,p):
    c = params[0] 
    phi = params[1:8]
    sigma2 = params[8]
    eps = np.random.normal(0,sigma2)
    ybuono = np.flip(y[-p:])
    forecast = []

    for i in range(h):
        first_stage = np.dot(ybuono, phi)
        forecast_t = c + first_stage + eps
        forecast.append(forecast_t)
    return forecast

cond_forecast = calculate_forecast(cond_estimate.x,y,8,7)
print(cond_forecast)

uncond_forecast = calculate_forecast(uncond_estimate.x,y,8,7)
print(uncond_forecast)





    

[0.0010216186012910434, 0.0010216186012910434, 0.0010216186012910434, 0.0010216186012910434, 0.0010216186012910434, 0.0010216186012910434, 0.0010216186012910434, 0.0010216186012910434]
[0.0006977752317427529, 0.0006977752317427529, 0.0006977752317427529, 0.0006977752317427529, 0.0006977752317427529, 0.0006977752317427529, 0.0006977752317427529, 0.0006977752317427529]
